# Introduction to Tests and Analysis of YOLOv11s Model for Pothole Detection

This documentation provides an overview of the tests and analysis conducted on the YOLOv11s model, which is designed for detecting potholes.

## Key Components

### Model Architecture
- **YOLOv11s**: A compact version of the YOLO model, designed for efficient and accurate object detection.

### Dataset
- **Pothole Images**: A collection of images containing various types of potholes used for training and testing the model.

### Evaluation Metrics
- **Precision**: The ratio of correctly predicted positive observations to the total predicted positives.
- **Recall**: The ratio of correctly predicted positive observations to all observations in the actual class.
- **F1 Score**: The weighted average of Precision and Recall.
- **Mean Average Precision (mAP)**: The average precision across all classes.

## Testing Procedure

1. **Data Preprocessing**: Images are resized, normalized, and augmented to improve model robustness.
2. **Model Training**: The YOLOv11s model is trained on the preprocessed dataset using a predefined training schedule.
3. **Validation**: The model's performance is validated on a separate validation set to tune hyperparameters.
4. **Testing**: The final model is tested on a test set to evaluate its performance using the aforementioned metrics.

## Analysis

- **Performance Metrics**: Detailed analysis of the model's precision, recall, F1 score, and mAP.
- **Error Analysis**: Examination of false positives and false negatives to identify common failure cases.
- **Inference Speed**: Measurement of the model's inference time to ensure real-time detection capabilities.

## Conclusion

The YOLOv11s model demonstrates effective pothole detection with a balance between accuracy and speed. The analysis highlights the model's strengths and areas for improvement, providing insights for further development and deployment in real-world scenarios.

In [11]:
import torch
import cv2
from ultralytics import YOLO


# Import the yolov11 model
try:
    model = YOLO("pothole_model.pt")  # Path to your model
    print("Model loaded successfully!")
except Exception as e:
    print(f"Failed to load model: {e}")

# Function to perform pothole detections
def pothole_detect(img_path):
    
    # Read the image
    img = cv2.imread(img_path)

    # Print the type of model to debug
    print(f"Type of model: {type(model)}")

    # Pass the image through the detection model and get the result
    detect_result = model(img)

    # Plot the detections
    detect_img = detect_result[0].plot()
    
    # Convert the image to RGB format
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)
    
    return detect_img

Failed to load model: PytorchStreamReader failed locating file data.pkl: file not found


In [ ]:

import os
import random
import matplotlib.pyplot as plt

# Define the directory where the custom images are stored
custom_image_dir = '/home/l1gthzao/Desktop/FAA/yolo_pothole_detection/dataset/test/images'

# Get the list of image files in the directory
image_files = os.listdir(custom_image_dir)

# Select 16 random images from the list
selected_images = random.sample(image_files, 16)

# Create a figure with subplots for each image
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

# Iterate over the selected images and plot each one
for i, img_file in enumerate(selected_images):
    
    # Compute the row and column index of the current subplot
    row_idx = i // 4
    col_idx = i % 4
    
    # Load the current image and run object detection
    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = pothole_detect(img_path)
    
    # Plot the current image on the appropriate subplot
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)

![Pothole Detection](/home/l1gthzao/Desktop/FAA/yolo_pothole_detection/results/yolov11s_50_epochs/__results___files/__results___22_1.png)

## Results

### Performance Metrics

![Performance Metrics](/home/l1gthzao/Desktop/FAA/yolo_pothole_detection/results/yolov11s_50_epochs/__results___files/__results___14_0.png)

#### Train Box Loss and Train Class Loss

The train box loss and class loss decrease over the number of epochs (iterations over the dataset), but it is clear that the model hasn't converged yet. Perhaps training for more epochs would lead to better results. Apart from that, the fact that both losses are decreasing is a good sign that the model is learning and not overfitting.

#### Metrics Precision and Recall

The precision and recall metrics increased over the number of epochs and reached a plateau near the end of the training. This is a good sign that the model is capable of detecting true positives without missing many of them.

#### Metrics mAP50 and mAP50-95 and DFL loss

The mAP50 is at a level near 0.8 and converging, which is a good result for this model. The mAP50-95 ended up near 0.5, which is a significant difference from the mAP50. This means that the model still struggles with bounding the plotholes with high precision (high IoU thresholds). 

Aligning it with the DFL loss, which is low, but still not converging (specially on training), we can infer that the model could've learned better how to bound the potholes with higher precision.

Solutions for this issue could include using data augmentation, since the model might not have seen enough examples of potholes in different conditions or adjusting anchor sizes so  that they better fit the potholes. It wouldn't be a good idea to increase the number of epochs, since a lot of metrics are already converging/near convergence.

#### Validation Loss Metrics

The validation loss metrics are all near convergence, but slighly higher than the training loss metrics (about 0.2 difference). Despite this deviation, we don't find it to be a significant issue, since the validation data is unseen by the model. If it was bigger, it would be a sign of overfitting.
